## توضیحات
در این فایل روش های مختلف یادگیری ماشین نظارتی بر روی دادها پیاده سازی شده اند و در نهایت دقیق ترین مدل برای پیش بینی احتمال موفقیت پرتاب از یک سکو مورد استفاده قرار گرفته است همچنین از روش گرید سرچ برای دستیابی به هایپر پارامتر های مودل نهایی استفاده شده است

In [ ]:
# فراخوانی کتابخانه های مورد نیاز
import piplite
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# دریاف داده ها
URL1 = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv"

URL2 = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_3.csv'
data = pd.read_csv(URL1)
X = pd.read_csv(URL2)
print(data.shape,X.shape)

In [ ]:
# جدا کردن لیبل 
y = data["Class"].to_numpy()

In [ ]:
# اسکیل کردن متغییر های پیشبینی کننده 
transform = preprocessing.StandardScaler()
X=transform.fit_transform(X)

In [ ]:
# تقسیم داده ها به دو گروه تست و آزمایش
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape)

In [ ]:
# تعریف تابع برای رسم ماتریس درهم ریختگی
def plot_confusion_matrix(y,y_predict):
    "this function plots the confusion matrix"
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax); #annot=True to annotate cells
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')
    ax.set_title('Confusion Matrix'); 
    ax.xaxis.set_ticklabels(['did not land', 'land']); ax.yaxis.set_ticklabels(['did not land', 'landed']) 
    plt.show() 

## مدل رگرسیون لوجستیک

In [ ]:
# تعریف پارامترهای مدل برای پیدا کردن اوپتیمال ترین هایپر پارامتر ها
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}

In [ ]:
# اجرای مدل 
parameters ={"C":[0.01,0.1,1],'penalty':['l2'], 'solver':['lbfgs']}# l1 lasso l2 ridge
lr=LogisticRegression()
logreg_cv = GridSearchCV(lr,parameters,cv=10)
logreg_cv.fit(X_train,y_train)

In [ ]:
# مشاهده بهترین اندازه پارامترهای مدل برازش شده
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

In [ ]:
# بررسی میزان دقت مدل
logreg_cv.score(X_test,y_test)

In [ ]:
# رسم ماتریس د رهم ریختگی
yhat=logreg_cv.predict(X_test)
plot_confusion_matrix(y_test,yhat)

## مدل SVM

In [ ]:
# تعریف پارامترهای مدل  برای پیدا کردن اوپتیمال ترین هایپر پارامتر ها
parameters = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 5),
              'gamma':np.logspace(-3, 3, 5)}
svm = SVC()

In [ ]:
# اجرای مدل 
svm_cv = GridSearchCV(svm,parameters,cv=10)
svm_cv.fit(X_train,y_train)

In [ ]:
# مشاهده بهترین اندازه پارامترهای مدل برازش شده
print("tuned hpyerparameters :(best parameters) ",svm_cv.best_params_)
print("accuracy :",svm_cv.best_score_)

In [ ]:
# بررسی میزان دقت مدل
svm_cv.score(X_test,y_test)

In [ ]:
# رسم ماتریس د رهم ریختگی
yhat=svm_cv.predict(X_test)
plot_confusion_matrix(y_test,yhat)

## مدل درخت تصمیم


In [ ]:
# تعریف پارامترهای مدل  برای پیدا کردن اوپتیمال ترین هایپر پارامتر ها
parameters = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

tree = DecisionTreeClassifier()

In [ ]:
# اجرای مدل 
tree_cv = GridSearchCV(tree,parameters,cv=10)
tree_cv.fit(X_train,y_train)

In [ ]:
# مشاهده بهترین اندازه پارامترهای مدل برازش شده
print("tuned hpyerparameters :(best parameters) ",tree_cv.best_params_)
print("accuracy :",tree_cv.best_score_)

In [ ]:
# بررسی میزان دقت مدل
tree_cv.score(X_test,y_test)

In [ ]:
# رسم ماتریس د رهم ریختگی
yhat = tree_cv.predict(X_test)
plot_confusion_matrix(y_test,yhat)

## همسایگی k روش 


In [ ]:
# تعریف پارامترهای مدل  برای پیدا کردن اوپتیمال ترین هایپر پارامتر ها
parameters = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

KNN = KNeighborsClassifier()

In [ ]:
# اجرای مدل 
knn_cv = GridSearchCV(KNN,parameters,cv=10)
knn_cv.fit(X_train,y_train)

In [ ]:
# مشاهده بهترین اندازه پارامترهای مدل برازش شده
print("tuned hpyerparameters :(best parameters) ",knn_cv.best_params_)
print("accuracy :",knn_cv.best_score_)

In [ ]:
# بررسی میزان دقت مدل
knn_cv.score(X_test,y_test)

In [ ]:
# رسم ماتریس د رهم ریختگی
yhat = knn_cv.predict(X_test)
plot_confusion_matrix(y_test,yhat)